In [3]:
import pandas as pd
import matplotlib
import requests
from bs4 import BeautifulSoup
import re

/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
#Grab Match Statistics from the match
def grab_match_stats(match_id):
    url = 'https://www.fotmob.com/match/{}'.format(match_id)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    #Grab HTML lines that matter
    scripts = soup.find_all('span')
    
    #Grab home and away team names
    home_team = soup.find_all('span', {'class': "css-er0nau-TeamName e3q4wbq4"})[0].text
    away_team = soup.find_all('span', {'class': "css-er0nau-TeamName e3q4wbq4"})[1].text
    
    #Grab home and away team scores
    home_goals = soup.find_all('span', {'class': "css-bw7eig-topRow"})[0].text[0]
    away_goals = soup.find_all('span', {'class': "css-bw7eig-topRow"})[0].text[-1]
    
    #Grab lines of HTML that hold stats data
    titles = soup.find_all('span', {'class': "title css-1s3albn-StatTitle e683amr4"})
    
    #Avoid these stats as it messes up the scraper (weird data types), but for the rest of the stats put them into the titles_new list
    titles_avoid = ['Shots', 'Passes', 'Discipline', 'Defence', 'Duels']
    titles_new = []
    for i in titles:
        if i.text not in titles_avoid:
            titles_new.append(i)
    titles_new
    
    #Structure of this scrape: HTML line has the name of the stat, with the 
    #home team's value of the stat in the line above and the away team's value in the line below
    data_dict = {}
    #Parse through titles
    for title in titles_new:
        #Parse through scripts
        for i, script in enumerate(scripts):
            #If the script is the title (stat) we are parsing through at the moment, 
            #grab the text above and below and throw it into a dictionary
            if title.text == script.text:
                data_dict[title.text] = [scripts[i - 1].text, scripts[i + 1].text]        
                break

    data_dict
    
    #Create Dataframe from this dictionary, adding home and away team names and goals
    df = pd.DataFrame.from_dict(data_dict)
    df.index = [home_team, away_team]
    df['goals'] = [home_goals, away_goals]
    
    return df

In [5]:
#Example
grab_match_stats('3780921')

,Expected goals (xG),Total shots,Big chances,Big chances missed,Accurate passes,Fouls committed,Offsides,Corners,Shots off target,Shots on target,...,Tackles won,Interceptions,Blocks,Clearances,Keeper saves,Duels won,Ground duels won,Aerial duels won,Successful dribbles,goals
Paris Saint-Germain,1.71,21,1,1,617 (91%),14,1,7,7,8,...,7 (47%),6,1,5,0,45,0,0 (0%),18 (62%),1
Real Madrid,0.26,3,0,0,435 (85%),12,0,1,2,0,...,10 (50%),11,6,11,8,45,0,4 (100%),9 (47%),0
